In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
%matplotlib inline

Importing Relevant Data only from Set

In [9]:
claims_data_df = pd.read_csv('appendectomy_44960.csv', index_col=[0])
claims_data_df

,Claim_Line_No,Place_of_Service_Cd,Rev_Cd,CPT4_ID,CPT4_Mod1_Cd,CPT4_Mod2_Cd,CPT4_Mod3_Cd,CPT4_Mod4_Cd,Service_Start_Dt,Units,...,NDC_Cd,Capitation_Flag,ER_Flag,Copay_Amt,Coinsurance_Amt,Deductible_Amt,Dental_Flag,Unit_Of_Measure_Cd,Unit_Of_Measure_Desc,Service_Start_Year
Claim_ID,,,,,,,,,,,,,,,,,,,,,
171144960,1,11,NaN,90834,,,,,2016-02-03,1,...,0,N,N,0.00,0.00,69.00,N,UN,Units,2016
306644960,1,11,NaN,97014,,,,,2016-06-06,1,...,0,N,N,15.56,2.85,0.00,N,UN,Units,2016
306644960,2,11,NaN,97035,,,,,2016-06-06,1,...,0,N,N,12.67,2.67,0.00,N,UN,Units,2016
306644960,3,11,NaN,97110,,,,,2016-06-06,1,...,0,N,N,6.77,6.37,0.00,N,UN,Units,2016
306644960,4,11,NaN,97140,,,,,2016-06-06,1,...,0,N,N,0.00,5.97,0.00,N,UN,Units,2016
168044960,1,11,NaN,93271,,,,,2016-03-09,1,...,0,N,N,0.00,0.00,0.00,N,UN,Units,2016
167944960,1,11,NaN,97110,GP,,,,2016-05-25,2,...,0,N,N,15.00,0.00,0.00,N,UN,Units,2016
167944960,2,11,NaN,97140,GP,,,,2016-05-25,1,...,0,N,N,0.00,0.00,0.00,N,UN,Units,2016
167944960,3,11,NaN,97014,GP,,,,2016-05-25,1,...,0,N,N,0.00,0.00,0.00,N,UN,Units,2016


In [10]:
claims_data_df['CPT4_ID'].nunique()

586

In [8]:
list(claims_data_df1.columns.values)

['Claim_Line_No',
 'Place_of_Service_Cd',
 'Rev_Cd',
 'CPT4_ID',
 'CPT4_Mod1_Cd',
 'CPT4_Mod2_Cd',
 'CPT4_Mod3_Cd',
 'CPT4_Mod4_Cd',
 'Service_Start_Dt',
 'Units',
 'Charge_Amt',
 'Prepaid_Amt',
 'Plan_Paid_Amt',
 'NDC_Cd',
 'Capitation_Flag',
 'ER_Flag',
 'Copay_Amt',
 'Coinsurance_Amt',
 'Deductible_Amt',
 'Dental_Flag',
 'Unit_Of_Measure_Cd',
 'Unit_Of_Measure_Desc',
 'Service_Start_Year']

Cleaning


In [ ]:
edited_df = medical_line_df.drop(['Dental_Flag', 'Service_Start_Year', 'Unit_Of_Measure_Desc', 'Unit_Of_Measure_Cd'], axis=1)
medical_line_edited

Engineering and Featurization 
Transform all of the non-numeric data into numeric data;
*get dummies
*masking
*boolean
*?

First Shitty Model
Initial Exploration of Features 


Tweakage of Model

Random Forest- Feature Importance/Decision Tree Evaluation

Quantile Regression- see Matt

Boosting-